# *Iris* flower classification with micrograd
In this notebook, you will implement a 2-layer (4-16-3) fully connected
feed-forward neural network to classify species of of *Iris* flowers, based on
four different measurements (sepal length, sepal width, petal length, and petal
width).

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import random
import micrograd.nn as nn
from micrograd.engine import Value
%matplotlib inline

np.random.seed(42)
random.seed(42)

First, let's setup our dataset. 

The "x" variables are the images, while the "y" variables are the ground truths.

In [ ]:
iris = load_iris()
train_x, test_x, train_y, test_y = train_test_split(iris.data, iris.target, test_size=0.05)
print(f"{train_x.shape=}\n{train_y.shape=}\n{test_x.shape=}\n{test_y.shape=}")

Now that we have our data loaded, we can initialize our model, using the
abstractions we wrote in `micrograd/nn.py` (imported above as `nn`).

Remember that we are looking to create a multi-layer perceptron, with one hidden
layer of dimension 16, an input layer of 4, and output layer of 3.

In [ ]:
model = nn.MLP(4, [16, 3])

Now, let's try evaluating our model on the first flower in our training set.

In [ ]:
model(train_x[0])

Now, of course, this returns total garbage, because our model is just running on
initialized random values for its weights. First, let's make sure that our model
is actually outputting a list of probabilities for each digit. We will use the
softmax function to do this:
$$\sigma(z)_i=\frac{\exp(z_i)}{\sum_{c=1}^C\exp(z_c)}$$
Where $C$ is number of classes (3). Implement this function below. Remember,
you are working with `Value`s, so you can only use the operations you defined
in `micrograd/engine.py`.

In [ ]:
def softmax(z: list[Value], C:int=10) -> list[Value]:
  pass # TODO

Now we can try evaluating the model again, with our new softmax function.

In [ ]:
softmax(model(train_x[0]))

This looks a little better, doesn't it? Of course, it's still not an accurate
prediction, but at least it is a prediction. Now we need to worry about training
the model. 

In order to train the model, we first need to find a way to measure how wrong
its predictions are, that is we need to define our cost (loss) function. We will
use cross entropy loss, which is defined by the following equation:
$$\ell (H(x),y)=-\log\frac{\exp(H(x)_{y})}{\sum_{c=1}^C\exp(H(x)_c)}$$
Where $x$ is the input image, $H$ is our model, $y$ is the ground truth
corresponding to $x$, and $C$ is the number of classes (3). But note that this
is the same as:
$$\ell (H(x),y)=-\log(\sigma(H(x))_y)$$
Where $\sigma$ is softmax from above.

Remember that since you are working with `Value`'s you only have access to the
operations defined in `micrograd/engine.py`.

In [ ]:
def loss(pred: list[Value], truth: int) -> Value:
  pass # TODO

Now that our loss function is defined, finally, we can write the code for
gradient descent.

In [ ]:
def argmax(arr: list[Value]) -> int:
  return arr.index(max(arr, key=lambda x: x.data))

for epoch in range(100):
  # forward
  scores = list(map(lambda x: model(x), train_x))
  losses = list(map(lambda tup: loss(tup[1], train_y[tup[0]]), enumerate(scores)))
  total_loss = sum(losses) * (1.0 / len(losses))

  # backward
  model.zero_grad()
  total_loss.backward()

  # update (sgd)
  learning_rate = (1.0 - 0.9*epoch/100) / 10
  for p in model.parameters():
    p.data -= learning_rate * p.grad

  if epoch % 10 == 0: print(f"Epoch {epoch}: Accuracy={sum([argmax(softmax(model(x))) == y for x, y in zip(test_x, test_y)])/len(test_y)}, Loss={total_loss.data}")

In [ ]:
print("Final accuracy:", sum([argmax(softmax(model(x))) == y for x, y in zip(test_x, test_y)])/len(test_y))